In [1]:
from typing import List, Dict, Literal
import json
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    # A new method
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    # A new method
    def reset(self) -> None:
        self._messages = []

In [5]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [6]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}}]

In [7]:
memory.reset()

In [8]:
memory.get_messages()

[]

In [9]:
def chat_with_tools(user_question:str=None, 
                    memory:Memory=None, 
                    model:str="gpt-4o-mini", 
                    temperature=0.0, 
                    tools=None)-> str:
    messages = [{"role": "user", "content": user_question}]
    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()        
    
    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools=tools, # Providing available tools to the model
    )
    
    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls # If the model decides to call a function
    
    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
    
    return ai_message

In [10]:
chat_with_tools(
    "2 to the power of -5?",
    model="gpt-3.5-turbo",
)

'1/32'

In [11]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [12]:
power(2, 3)

8

In [13]:
power(2, -5)

0.03125

In [15]:
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentatiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [16]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [17]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}}]

In [18]:
ai_message = chat_with_tools(
    "2 to the power of -5?",
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [19]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_3IIXf45IfT69oRderdW0W7Mo', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]}]

In [21]:
memory.last_message()['tool_calls']

[ChatCompletionMessageToolCall(id='call_3IIXf45IfT69oRderdW0W7Mo', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]

In [22]:
tool_call_id = memory.last_message()['tool_calls'][0].id
tool_call_id

'call_3IIXf45IfT69oRderdW0W7Mo'

In [23]:
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
args

{'base': 2, 'exponent': -5}

In [24]:
result = power(args["base"], args["exponent"])
result

0.03125

In [25]:
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)

In [26]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_3IIXf45IfT69oRderdW0W7Mo', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_3IIXf45IfT69oRderdW0W7Mo'}]

In [27]:
ai_message = chat_with_tools(
    model="gpt-3.5-turbo",
    tools=tools,
    memory=memory,
)

In [28]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant", 'tool_calls': {}},
 {'role': 'user', 'content': '2 to the power of -5?', 'tool_calls': {}},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_3IIXf45IfT69oRderdW0W7Mo', function=Function(arguments='{"base":2,"exponent":-5}', name='power'), type='function')]},
 {'role': 'tool',
  'content': '0.03125',
  'tool_call_id': 'call_3IIXf45IfT69oRderdW0W7Mo'},
 {'role': 'assistant',
  'content': '2 to the power of -5 is 0.03125.',
  'tool_calls': None}]